In [1]:
from mycelia.shared.model import load_base_model
from mycelia.shared.modeling_moe import get_base_tokenizer
from mycelia.config import Config
from mycelia.shared.datasets import get_dataloader
from mycelia.shared.logging import configure_logging

configure_logging()

config = Config()
config.model.device = "cuda:2"
model, em = load_base_model(config = config, rank = 0)
tokenizer = get_base_tokenizer(config)
dataloader = get_dataloader(
    config,
    rank= config.data.rank,
    world_size= config.data.world_size,
    tokenizer=tokenizer,
)
data_iter = iter(dataloader)

/home/isabella/crucible/subnet-MoE/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


2025-10-03 07:39:27 [info     ] rank 0: Checkpoint path checkpoints/centralised not found, starting from scratch [mycelia.shared.checkpoint]


Loading checkpoint shards: 100%|██████████| 7/7 [00:06<00:00,  1.05it/s]


Instantiating DeepseekAttention without passing `layer_idx` is not recommended and will to errors during the forward call, if caching is used. Please make sure to provide a `layer_idx` when creating this class.
2025-10-03 07:40:27 [info     ] em init                      [mycelia.shared.expert_manager]
2025-10-03 07:40:27 [info     ] em get expert layer          [mycelia.shared.expert_manager]
2025-10-03 07:40:27 [info     ] em get expert layer 1        [mycelia.shared.expert_manager]
2025-10-03 07:40:27 [info     ] Detected expert layerss: [1, 3, 5, 7, 9, 11, 13, 15, 17, 19, 21, 23, 25, 27] [mycelia.shared.expert_manager]
2025-10-03 07:40:27 [info     ] seed 0 Expert group assignment for layer 27: {0: [1, 4, 5, 3], 1: [7, 2, 6, 0]} [mycelia.shared.expert_manager]


Loading checkpoint shards: 100%|██████████| 7/7 [00:07<00:00,  1.05s/it]


In [ ]:
from transformers import (
    AutoConfig,
    AutoTokenizer,
    LlamaConfig,
    LlamaForCausalLM,
    OlmoForCausalLM,
    AutoModelForCausalLM,
    PretrainedConfig
)
org_model = AutoModelForCausalLM.from_pretrained(config.model.model_path, trust_remote_code=True)
output = org_model(**batch_device)

Loading checkpoint shards: 100%|██████████| 7/7 [01:48<00:00, 15.56s/it]


In [21]:

batch = next(data_iter)  

batch_device = {}
for key in batch.keys():
    batch_device[key] = batch[key].to(model.device)

In [22]:
model.train()

OptimizedModule(
  (_orig_mod): MyceliaMoE(
    (model): DeepseekV3Model(
      (embed_tokens): Embedding(102400, 2048)
      (layers): ModuleList(
        (0): DeepseekV3DecoderLayer(
          (self_attn): DeepseekAttention(
            (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
            (k_proj): Linear(in_features=2048, out_features=2048, bias=False)
            (v_proj): Linear(in_features=2048, out_features=2048, bias=False)
            (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
            (rotary_emb): DeepseekRotaryEmbedding()
          )
          (mlp): DeepseekV3MLP(
            (gate_proj): Linear(in_features=2048, out_features=10944, bias=False)
            (up_proj): Linear(in_features=2048, out_features=10944, bias=False)
            (down_proj): Linear(in_features=10944, out_features=2048, bias=False)
            (act_fn): SiLU()
          )
          (input_layernorm): DeepseekV3RMSNorm((2048,), eps=1e-06)
          (po

In [23]:
import torch
with torch.amp.autocast("cuda", dtype=torch.float16):
    output = model(**batch_device)

2025-10-03 07:47:41 [info     ] moe                          [mycelia.shared.modeling_moe] positional_args=(tensor(34595.0781, device='cuda:2', grad_fn=<SumBackward0>),)


2025-10-03 07:47:41 [info     ] moe                          [mycelia.shared.modeling_moe] positional_args=(tensor(3826.8367, device='cuda:2', grad_fn=<SumBackward0>),)
2025-10-03 07:47:41 [info     ] moe                          [mycelia.shared.modeling_moe] positional_args=(tensor(-50449.9297, device='cuda:2', grad_fn=<SumBackward0>),)
2025-10-03 07:47:41 [info     ] moe                          [mycelia.shared.modeling_moe] positional_args=(tensor(-60080.7812, device='cuda:2', grad_fn=<SumBackward0>),)
2025-10-03 07:47:41 [info     ] moe                          [mycelia.shared.modeling_moe] positional_args=(tensor(-104194.4766, device='cuda:2', grad_fn=<SumBackward0>),)
2025-10-03 07:47:41 [info     ] moe                          [mycelia.shared.modeling_moe] positional_args=(tensor(-105743.1250, device='cuda:2', grad_fn=<SumBackward0>),)
2025-10-03 07:47:41 [info     ] moe                          [mycelia.shared.modeling_moe] positional_args=(tensor(-125391.1328, device='cuda:2',

OutOfMemoryError: CUDA out of memory. Tried to allocate 160.00 MiB. GPU 2 has a total capacity of 47.40 GiB of which 79.62 MiB is free. Including non-PyTorch memory, this process has 47.31 GiB memory in use. Of the allocated memory 46.28 GiB is allocated by PyTorch, and 732.86 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [20]:
output.loss

tensor(20.3762, device='cuda:2', grad_fn=<CompiledFunctionBackward>)

In [7]:
output.loss.backward()

In [13]:
for p in model.parameters():
    if p.grad is not None:
        p.grad.zero_()
        
model.zero_grad()

In [15]:
from mycelia.miner.evaluate import evaluate_model
val_metric = evaluate_model(rank = 0, step = 0, model = model, eval_dataloader = dataloader, device = "cuda:2")

2025-10-03 07:46:07 [info     ] moe                          [mycelia.shared.modeling_moe] positional_args=(tensor(16162.1484, device='cuda:2'),)
2025-10-03 07:46:10 [info     ] moe                          [mycelia.shared.modeling_moe] positional_args=(tensor(26042.5508, device='cuda:2'),)
2025-10-03 07:46:10 [info     ] moe                          [mycelia.shared.modeling_moe] positional_args=(tensor(-7818.3735, device='cuda:2'),)
2025-10-03 07:46:11 [info     ] moe                          [mycelia.shared.modeling_moe] positional_args=(tensor(5607.7100, device='cuda:2'),)
2025-10-03 07:46:11 [info     ] moe                          [mycelia.shared.modeling_moe] positional_args=(tensor(-64982.1953, device='cuda:2'),)
2025-10-03 07:46:11 [info     ] moe                          [mycelia.shared.modeling_moe] positional_args=(tensor(-71738.6250, device='cuda:2'),)
2025-10-03 07:46:11 [info     ] moe                          [mycelia.shared.modeling_moe] positional_args=(tensor(-75970.3

W1003 07:46:13.088000 1598998 torch/_dynamo/convert_frame.py:1016] [25/8] torch._dynamo hit config.recompile_limit (8)
W1003 07:46:13.088000 1598998 torch/_dynamo/convert_frame.py:1016] [25/8]    function: 'torch_dynamo_resume_in___call___at_698' (/home/isabella/crucible/subnet-MoE/.venv/lib/python3.10/site-packages/structlog/dev.py:698)
W1003 07:46:13.088000 1598998 torch/_dynamo/convert_frame.py:1016] [25/8]    last reason: 25/7: ___stack0[0] == '\x1b2025-10-03 07:46:11\x1b'    
W1003 07:46:13.088000 1598998 torch/_dynamo/convert_frame.py:1016] [25/8] To log all recompilation reasons, use TORCH_LOGS="recompiles".
W1003 07:46:13.088000 1598998 torch/_dynamo/convert_frame.py:1016] [25/8] To diagnose recompilation issues, see https://pytorch.org/docs/main/torch.compiler_troubleshooting.html.


2025-10-03 07:46:13 [info     ] moe                          [mycelia.shared.modeling_moe] positional_args=(tensor(8551.3574, device='cuda:2'),)
2025-10-03 07:46:13 [info     ] moe                          [mycelia.shared.modeling_moe] positional_args=(tensor(36504.3164, device='cuda:2'),)
2025-10-03 07:46:13 [info     ] moe                          [mycelia.shared.modeling_moe] positional_args=(tensor(-11761.8105, device='cuda:2'),)
2025-10-03 07:46:13 [info     ] moe                          [mycelia.shared.modeling_moe] positional_args=(tensor(-12132.3555, device='cuda:2'),)
2025-10-03 07:46:13 [info     ] moe                          [mycelia.shared.modeling_moe] positional_args=(tensor(-42484.4336, device='cuda:2'),)
2025-10-03 07:46:13 [info     ] moe                          [mycelia.shared.modeling_moe] positional_args=(tensor(-12464.5361, device='cuda:2'),)
2025-10-03 07:46:13 [info     ] moe                          [mycelia.shared.modeling_moe] positional_args=(tensor(-16447

In [16]:
val_metric

{'val_loss': 20.83195400238037, 'val_aux_loss': 0.0}